A1.4 Seleccion de caracteristicas
Jesus Esteban Bustamante Apodaca

I) Importa los datos del archivo “Vino Tinto.csv” a tu ambiente de trabajo. Este archivo lo
encontrarás en la misma página donde descargaste esta plantilla. Revisa las dimensiones
del data frame e imprime en consola tanto dichas dimensiones como las primeras 5 filas
de datos.

In [2]:
import pandas as pd
import numpy as np

ds = pd.read_csv("A1.4 Vino Tinto.csv")
print("Dimensiones del Data Frame: ", ds.shape)
print("Primeras 5 filas: ")
print(ds.head(5))


Dimensiones del Data Frame:  (1599, 12)
Primeras 5 filas: 
   acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
0         7.4           0.70          0.00             1.9     0.076   
1         7.8           0.88          0.00             2.6     0.098   
2         7.8           0.76          0.04             2.3     0.092   
3        11.2           0.28          0.56             1.9     0.075   
4         7.4           0.70          0.00             1.9     0.076   

   dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  alcohol  \
0                11.0                34.0    0.9978  3.51      0.56      9.4   
1                25.0                67.0    0.9968  3.20      0.68      9.8   
2                15.0                54.0    0.9970  3.26      0.65      9.8   
3                17.0                60.0    0.9980  3.16      0.58      9.8   
4                11.0                34.0    0.9978  3.51      0.56      9.4   

   calidad  
0        5  
1

II) Separa el data frame en datos de entrenamiento y datos de prueba con una proporción
80/20. Es decir, el 80% de los datos se usarán para entrenar el modelo y el resto para
validar sus resultados. Asegúrate que la partición sea aleatoria, no es una buena práctica
simplemente tomar las primeras observaciones para entrenar y las últimas para probar.
Imprime en pantalla las dimensiones de ambos conjuntos de datos. Revisa y asegúrate
que la cantidad de observaciones de ambos conjuntos de datos sumen a la cantidad de
datos original.

In [3]:
from sklearn.model_selection import train_test_split
ds_entrenamiento, ds_prueba = train_test_split(ds, test_size=0.2, random_state=42)
print("Dimensiones de entrenamiento:", ds_entrenamiento.shape)
print("Dimensiones de prueba:", ds_prueba.shape)
print("Total de observaciones:", len(ds))
print("Suma de entrenamiento y prueba:", len(ds_entrenamiento) + len(ds_prueba))

Dimensiones de entrenamiento: (1279, 12)
Dimensiones de prueba: (320, 12)
Total de observaciones: 1599
Suma de entrenamiento y prueba: 1599


III) Genera la metodología de selección hacia adelante e imprime en consola los índices o
los nombres de las características seleccionadas. Para realizar este proceso, te
SC3314 – Inteligencia Artificial Universidad de Monterrey Dr. Antonio Martínez Torteya
recomiendo que utilices la función “SequentialFeatureSelector” de la librería
“mlxtend.feature_selection”, en este enlace encontrarás más información sobre la misma.
Lo más probable es que cuando hayas descargado Anaconda, esta librería no se haya
incluido en la distribución, por lo que deberás instalarla manualmente; al final de las
instrucciones de la actividad te indico cómo hacerlo. Aquí te dejo una descripción de los
parámetros que te recomiendo usar:

a. estimator. Un modelo de regresión lineal. Te recomiendo usar la función LinearRegression de la librería sklearn.linear_model.

b. k_features. Se puede seleccionar la cantidad de variables de salida que se desean, pero te recomiendo mejor usar un rango, y que el algoritmo determine el número adecuado. Por ejemplo, puedes definir el parámetro como (2,8), si te interesa que el método seleccione entre 2 y 8 variables.

c. forward. Determina si se hace selección hacia adelante (True) o hacia atrás (False); en este caso queremos hacer selección hacia adelante.

d. scoring. La métrica que se usará para determinar si un modelo es mejor que otro,te recomiendo definirla como ‘r2’ para usar la R cuadrada.

e. cv. Si se desea realizar validación cruzada, y cuántas instancias de la misma. Terecomiendo definir este parámetro como 10.

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
# Separar las variables independientes y la variable dependiente
# Todas las columnas excepto la de calidad son las independientes, y calidad seria la dependiente pues esta seria la variable de salida
x = ds.drop('calidad', axis=1)
y = ds['calidad']

modelo = LinearRegression()
sfs = SFS(modelo, k_features=(2,8), forward=True,scoring='r2',cv=10)

sfs.fit(x, y)
selected_features = list(sfs.k_feature_names_)
print("Características seleccionadas (Nombres):", selected_features)

Características seleccionadas: ['acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']


IV) Entrenar un modelo que solamente contenga las variables seleccionadas, predecir la
respuesta en las observaciones de prueba y medir la capacidad de predicción del modelo
usando la R cuadrada, imprimiendo dicho valor en consola. Para el primer paso,
simplemente necesitas usar la función fit en el modelo de regresión lineal creado
previamente, asegurándote de no introducir toda la información de X, sino solo de las
variables seleccionadas. Para realizar las predicciones, puedes usar la función predict en
los datos de prueba, pero recuerda para dichos datos también seleccionar solo las
variables de interés. Para el último paso, te recomiendo usar la función r2_score de
sklearn.metrics.

In [5]:
from sklearn.metrics import r2_score

X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(x, y, test_size=0.2, random_state=42)
X_entrenamiento_seleccionado = X_entrenamiento[selected_features]
X_prueba_seleccionado = X_prueba[selected_features]

modelo.fit(X_entrenamiento_seleccionado,y_entrenamiento)
y_prediccion = modelo.predict(X_prueba_seleccionado)
r2 = r2_score(y_prueba,y_prediccion)
print("R cuadrada en el conjunto de prueba:", r2)

R cuadrada en el conjunto de prueba: 0.4012628835440285


V) Realizar un proceso de selección hacia atrás a partir de las variables seleccionadas por
el método de selección hacia adelante e imprimir en consola los índices o nombres de las
variables seleccionadas. Para realizar este proceso, te recomiendo usar la misma función
del paso 3, pero definiendo ahora forward=False. También te recomiendo especificar una
menor cantidad de variables posibles, por ejemplo: k_features=(2,5).

In [ ]:
sbs = SFS(modelo, k_features=(2, 5), forward=False, floating=False, scoring='r2', cv=10)
sbs = sbs.fit(X_entrenamiento_seleccionado,y_entrenamiento)

sbs_selected_features = list(sbs.k_feature_names_)
print('Características seleccionadas (Nombres) - Selección hacia atrás:', sbs_selected_features)

Características seleccionadas - Selección hacia atrás: ['acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol']


VI) Repetir el paso 4, pero para un modelo que contenga solamente las variables
seleccionadas en el paso 5. Imprime en pantalla un breve texto que describa tu opinión
sobre la diferencia en R cuadrada medida entre los modelos de los pasos 4 y 6, ¿cuál
modelo consideras que es mejor? ¿Por qué?

In [8]:
X_entrenamiento_seleccionado_sbs = X_entrenamiento[sbs_selected_features]
X_prueba_seleccionado_sbs = X_prueba[sbs_selected_features]

modelo.fit(X_entrenamiento_seleccionado_sbs, y_entrenamiento)
y_prediccion_sbs = modelo.predict(X_prueba_seleccionado_sbs)
r2_sbs = r2_score(y_prueba, y_prediccion_sbs)
print("R cuadrada en el conjunto de prueba (Selección hacia atrás):", r2_sbs)

R cuadrada en el conjunto de prueba (Selección hacia atrás): 0.3958889666765396


Considerando los resultados en las R^2 de la seleccion hacia adelante y hacia atras, los resultados mostraron que la R cuadrada de la seleccion hacia adelante es mayor al de hacia atras, por lo que indica que es mejor en la predicción de la calidad del vino. Esto se debe a que una R cuadrada más alta implica una mayor capacidad del modelo para explicar la variabilidad de la calidad del vino.